In [1]:
import nni
# import autokeras as ak
# from keras.layers import Dense, LSTM, Dropout
# from keras.layers.embeddings import Embedding
# from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
import reber
RANDOM_STATE = 42
PADDING_IDX = 0

Using TensorFlow backend.


In [5]:
r = reber.ReberGenerator(max_length=15)
X, y = r.make_data(total_num_rows=3000)
_, word_len = X.shape
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.1, random_state=RANDOM_STATE
)

In [6]:
class Net():
    def __init__(self):
        super(Net, self).__init__()
        

In [ ]:
## nni
 

In [8]:
num_features = len(r._reber_letters) + 1 # include padding token
# TODO: how to mask in the embedding
input_node = ak.Input()
output_node = ak.Embedding(max_features=num_features)(input_node)
output_node = ak.RNNBlock(
    bidirectional=False,
    # num_layers=1,
    layer_type='gru'
)(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    max_trials=2
)
clf.fit(X_train.to_numpy(), y_train.to_numpy(), verbose=2, epochs=2)

INFO:tensorflow:Reloading Oracle from existing project ./auto_model/oracle.json
INFO:tensorflow:Reloading Tuner from ./auto_model/tuner0.json
INFO:tensorflow:Oracle triggered exit
Train for 57 steps, validate for 12 steps
Epoch 1/2
57/57 - 6s - loss: 0.6930 - accuracy: 0.5194 - val_loss: 0.6859 - val_accuracy: 0.5444
Epoch 2/2
57/57 - 3s - loss: 0.6931 - accuracy: 0.5144 - val_loss: 0.6873 - val_accuracy: 0.5444


In [ ]:
model = clf.export_model()

In [13]:
model.save('my_model.h5')

In [25]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=1,
    batch_size=5000
)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 80000 samples, validate on 20000 samples
Epoch 1/2
80000/80000 [==============================] - 17s 217us/step - loss: 0.6903 - accuracy: 0.5657 - val_loss: 0.6851 - val_accuracy: 0.4983
Epoch 2/2
80000/80000 [==============================] - 18s 224us/step - loss: 0.6665 - accuracy: 0.5379 - val_loss: 0.6395 - val_accuracy: 0.6458


In [6]:
strs = [
    "BPBTSSXXVVEPE", # true reber
    "BPBTSSSSXXVPXTTTVPSETE", # second and second to last not the same, but reber inbetween
    "BPBPTTTTTVPBTTTTVPSEPT", # second and second to last the same, but not reber in middle
    "BBBBBBBBBBB",
    "XXXXXXXXXXXXXX",
    "TTTTTTTTTTTTTTTTTTT"
]
x = np.array([
        r.encode_as_padded_ints(s) for s in strs
    ])
dataset = clf._process_x(x, False)
dataset = dataset.batch(6)
model = clf.tuner.get_best_model()
model.predict(x)

array([[0.46915156],
       [0.4183758 ],
       [0.41421646],
       [0.46194988],
       [0.44758365],
       [0.43123493]], dtype=float32)

Clearly, XXXXXXXXXX is not in the reber grammar, and yet the model predicted with 98.6% certainty that it was. This means that I need to make my examples of non-reber much more random.

In [24]:
## autokeras
embedding_vector_length = len(r._reber_letters) # TODO: what should this be?
num_neurons = 60
model = Sequential()
Embedding
model.add(
    Embedding(
        embedding_vector_length + 1,
        embedding_vector_length,
        input_length=word_len,
        mask_zero=True
    )
)
model.add(LSTM(num_neurons))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])